## ВМА. Лабораторная 7. Отчет
### Гамезо Валерия

### Постановка задачи

Вычислить вещественные корни собственного многочлена четвертой степени $P(\lambda)$, полученного из канонической формы Фробениуса лабораторной работы «Метод Данилевского». 

<ol>
    <li>
        Произвести отделение корней многочлена $P(\lambda)$.
        Для определения промежутков монотонности функции $P(\lambda)$ решить уравнение $P'(\lambda) = 0$: 
        <ol>
            <li>методом простой итерации;</li>
            <li>методом Ньютона</li>
        </ol>
        Предварительно произвести отделение корней многочлена $P'(\lambda)$.
    </li>
    <li>
        Приближенно вычислить вещественные корни собственного многочлена:
        <ol>
            <li>применить метод деления отрезка пополам;</li>
            <li>применить метод Ньютона.</li>
        </ol>
    </li>
</ol>

### Входные данные 

$$ P(\lambda) = x^4 - 52 x^3 + 813 x^2 + 291468 x - 841577$$

### Листинг программы

Абстрактный родтельский класс метода

```c++
class Method {
 protected:
  std::vector<float> function;

  float multiply(std::vector<float>&, float); // функция получения значения уравнения f(x) при f, 
                                              // заданном вектором коэффициэнтов a_i
};

float Method::multiply(std::vector<float>& f, float x) {
  float ans = f[f.size() - 1];

  for (int i = f.size() - 1; i > 0; --i) {
    ans = ans * x + f[i - 1];
  }

  return ans;
}
```

Класс метода Ньютона

```c++
class NewtonsMethod : public Method {
 public:
  NewtonsMethod(std::initializer_list<float>); // инициализация f(x) при помощи списка коэффициэнтов, записанных слева направо

  float getFuncVal(float); // получение значения функции f(x)
  float getDerivVal(float); // получение значения функции f'(x)

  float getX(float); // получение корня уравнения f(x) = 0 при заданном приближении x_0
 private:
  const float E = 1e-9;

  std::vector<float> derivative; // вектор коэффициентов производной исходной функции, инициализируется в конструкторе
};

NewtonsMethod::NewtonsMethod(std::initializer_list<float> coefs) {
  int size = coefs.size();
  function.resize(size);
  derivative.resize(size - 1);

  for (int i = 0, k = size - 1; i < size; ++i, k--) { // инициализация вектора коэффициентов функции f(x)
    function[k] = *(coefs.begin() + i);
  }

  for (int i = size - 1; i > 0; --i) { // инициализация вектора коэффициентов f'(x)
    derivative[i - 1] = function[i] * i;
  }
}

float NewtonsMethod::getFuncVal(float x) {
  return multiply(function, x);
}

float NewtonsMethod::getDerivVal(float x) {
  return multiply(derivative, x);
}

float NewtonsMethod::getX(float x0) {
  float xk = x0;
  float xkPlus = xk - getFuncVal(xk) / getDerivVal(xk);

  while(fabs(xk - xkPlus) > E) { // выполнение цикла до получения корня заданной константой точности 
    xk = xkPlus;
    xkPlus = xk - getFuncVal(xk) / getDerivVal(xk); // получение значения x_{k + 1} = x_k - f(x_k) / f'(x_k)
  }

  return xkPlus;
}
```

Класс метода бисекции

```c++
class BisectionMethod : public Method{
 public:
  BisectionMethod(std::initializer_list<float>);

  float getX(float, float);
 private:
  const float E = 2e-6;
};

BisectionMethod::BisectionMethod(std::initializer_list<float> coefs) {
  int size = coefs.size();
  function.resize(size);

  for (int i = 0, k = size - 1; i < size; ++i, k--) {
    function[k] = *(coefs.begin() + i);
  }
}

float BisectionMethod::getX(float a, float b) {
  if (multiply(function, a) * multiply(function, b) > 0) throw std::invalid_argument("Invalid range"); // проверка 
                                                                            // начального промежутка на валидность

  float m;

  while (b - a > 2 * E) {
    m = (a + b) / 2;
    float fM = multiply(function, m);
    if (fabs(fM) < E) return m;

    (multiply(function, a) * fM < 0) ? b = m : a = m;
  }

  return (a + b) / 2;
}
```

Класс метода простой итерации для нелинейных уравнений

```c++
class IterationMethod: public Method{
 public:
  IterationMethod(std::initializer_list<float>, float); // инициализации вектора коэффициентов функции phi(x)
                                                        // при помощи вектора коэффициентов функии f(x) и функции psi(x), 
                                                        // заданной константой c

  float getX(float);
 private:
  const float E = 1e-9;
};

IterationMethod::IterationMethod(std::initializer_list<float> coefs, float c) {
  int size = coefs.size();
  function.resize(size);

  for (int i = 0, k = size - 1; i < size; ++i, k--) { // инициализации вектора коэффициентов функции phi(x)
    function[k] = *(coefs.begin() + i) * c;
  }

  function[1]++;
}

float IterationMethod::getX(float x0) {
  float xk = x0;
  float xkPlus = multiply(function, xk);

  while (fabs(xk - xkPlus) > E) {
    xk = xkPlus;
    xkPlus = multiply(function, xk);
  }

  return xkPlus;
}
```

### Ход вычисления

Для отделения корней и выделения промежутков применим графический способ, представляя исходную функцию $f(x) = 0$ в виде болле простых $f_1(x) = f_2(x)$, где $f_1(x) = x^4$, а $f_2(x) = 52 x^3 - 813 x^2 - 291468 x + 841577$.

<img src = "initFunc.jpeg">

На графике можно заметить, что у нас имеется два вещественных корня, один из которых находится на промежутке $[-55, -50]$, а второй $- \; [0, 5]$.

Для определения промежутков монотонности решим уравнение $P'(\lambda) = 0$. $P'(\lambda) = 4 x^3 - 156 x^2 + 1626 x + 291468$. Предварительно отделим существующие вещественные корни, используя графический способ. Представим в $P'(\lambda)$ виде более простых $f_1(x), f_2(x) \; | \; f_1(x) = 2 x^3, f_2(x) = 78 x ^2 - 813 x - 145734$ так что $f_1(x) = f_2(x)$.

<img src = "derivFunc.jpeg">

По графику видно, что существует единсвенный вещественный корень, находящийся в промежутке $[-32, -28]$.

#### Вычисление корней уравнения производной методом простой итерации для нелинейных уравнений

Вичислим корень уравнения $P'(\lambda) = 0$ при помощи метода простой итерации. Для этого приведем уравнение к виду $x = \varphi(x)$, пригодному для итераций. Пусть $x = x + \psi(x) * \varphi(x)$, где $\psi(x) = (-1 \cdot 10^{-4}) - \text{const}$. Убедимся, что полученная функция удовлетворяет условию сходимости $|\varphi'(x)| < 1$ на отрезке $[-32, -28]$: 

<img src = "iterFunc.jpeg">

Корень, полученный при помощи метода простой итерации для нелинейных уравнений, равен: $-29.73616790771$, $\varepsilon = 1 \cdot 10^{-9}$.

#### Вычисление корней уравнения производной методом Ньютона

Корень, полученный при помощи метода Ньютона, равен: $-29.73616790771$, $\varepsilon = 1 \cdot 10^{-9}$.

В итоге исходная функция имеет следующие промежутки монотонности:

$$
\begin{array}{ccc}
    (-\infty, -29.73...) & \{-29.73616...\} & (-29.73..., +\infty) \\
    f(x) \downarrow & \text{global minimum} & f(x) \uparrow
\end{array}
$$

Следовательно, мы уточнили, что на промежутках $[-55, -50]$ и $[0, 5]$ исходная функция является монотонной.

#### Нахождение корней уравнения методом бисекции

Пусть начальные точки $x_0, x_1$ для первого вещественного корня уравнения равны конечным точкам промежутка $[-55, -50], \; x_0 = -55, \; x_1 = -50$. Тогда $f(x_0) = 3389133, \; f(x_1) = -632477 \implies f(x_0)f(x_1) < 0$. Следовательно, с условием, что на данном промежутке функция является монотонной, мы имеем единственное решение.

Первый корень, полученный при помощи метода бисекции, равен: $-50.90182495117$, $\varepsilon = 1 \cdot 10^{-5}$.

Пусть начальные точки $x_0, x_1$ для второго вещественного корня уравнения равны конечным точкам промежутка $[0, 5], \; x_0 = 0, \; x_1 = 5$. Тогда $f(x_0) = -841577, \; f(x_1) = 630213 \implies f(x_0)f(x_1) < 0$. Следовательно, с условием, что на данном промежутке функция является монотонной, мы имеем единственное решение.

Второй корень, полученный при помощи метода бисекции, равен: $2.86840081215$, $\varepsilon = 1 \cdot 10^{-5}$.

#### Нахождение корней уравнения методом Ньютона

Корни, полученные при помощи метода Ньютона: $ x_1 = -50.90182113647, \; x_2 = 2.86840176582, \; \varepsilon = 1 \cdot 10^{-9}$.